In [1]:
import numpy as np 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as T
from torch.utils.data import DataLoader, Subset
import matplotlib as plt 
import matplotlib_inline.backend_inline

matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [5]:
transform = T.Compose([T.ToTensor(), T.Normalize([.5,.5,.5],[.5,.5,.5])])
trainset = torchvision.datasets.CIFAR10(root = './data' , train =  True , download = True , transform = transform)
devset = torchvision.datasets.CIFAR10(root = './data' , train =  True , download = True , transform = transform)

randidx = np.random.permutation(100000)
devset = Subset(devset,randidx[:6000])
testset = Subset(devset,randidx[6000:])

batchsize = 32
train_loader = DataLoader(trainset,batch_size=batchsize , shuffle=True,drop_last=True)
dev_loader = DataLoader(devset , batch_size=batchsize)
test_loader = DataLoader(testset , batch_size=len(testset))


100%|██████████| 170498071/170498071 [00:55<00:00, 3051017.73it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [8]:
print(len(trainset))
print('\nData categories:')
print(trainset.classes)

X,y = next(iter(train_loader))
print( X.data.shape)

50000

Data categories:
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
torch.Size([32, 3, 32, 32])


In [24]:
def myNet(printtoggle = False):

    class cnn(nn.Module):
        def __init__(self,printtoggle):
            super().__init__()

            self.print = printtoggle

            self.conv1 = nn.Conv2d(3,64,3 , padding=1)
            self.bnorm1 = nn.BatchNorm2d(64)


            self.conv2 = nn.Conv2d(64,128,3)
            self.bnorm2 = nn.BatchNorm2d(128)

            self.conv3 = nn.Conv2d(128,256,3)
            self.bnorm3 = nn.BatchNorm2d(256)

            self.fc1 = nn.Linear(2*2*256,128)
            self.fc2 = nn.Linear(128,64)
            self.fc3 = nn.Linear(64,10)


        def forward(self,x):
            if self.print: print(f'Input: {list(x.shape)}')

            x = F.max_pool2d(self.conv1(x),2)
            x = F.leaky_relu(self.bnorm1(x))
            
            x = F.max_pool2d(self.conv2(x),2)
            x = F.leaky_relu(self.bnorm2(x))

            x = F.max_pool2d(self.conv3(x),2)
            x = F.leaky_relu(self.bnorm1(x))
            x = x.view(-1, 256 * 2 * 2)


            # nunits = x.shape.numel()/x.shape[0]
            # x = x.view(-1,int(nunits))
            if self.print: print(f'Vectorized: {list(x.shape)}')


            x = F.leaky_relu(self.fc1(x))
            x = F.dropout(x, p=0.5 , training = self.training)
            x = F.leaky_relu(self.fc2(x))
            x = F.dropout(x,p=.5,training=self.training) # training=self.training means to turn off during eval mode
            x = self.fc3(x)
            if self.print: print(f'Final output: {list(x.shape)}')

    net = cnn(printtoggle)
    lossfun = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(),lr = 0.001, weight_decay = 1e-5)

    return net , lossfun , optimizer










In [25]:
net,lossfun,optimizer = myNet(True)

X,y = next(iter(train_loader))
yHat = net(X)
print('\nOutput size:')
print(yHat.shape)
loss = lossfun(yHat,torch.squeeze(y))
print(' ')
print('Loss:')
print(loss)

Input: [32, 3, 32, 32]


RuntimeError: running_mean should contain 256 elements not 64